In [1]:
from collections import defaultdict, Counter, deque, OrderedDict
from intervaltree import Interval, IntervalTree
import itertools
import functools
import math
import numpy as np
import re
from typing import Callable, TypeVar

np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [56]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result

day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [5]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

def day4(data):
    result, cards = 0, Counter()
    for i, line in enumerate(data):
        matches = line[0].intersection(line[1])
        if matches:
            result += 2**(len(matches)-1)
        for n in range(len(matches)):
            cards[i+1+n] += cards[i]+1
    return result, cards.total() + len(data)

day4(data4)


(25651, 19499881)

In [55]:
debug=False

def day5_maps(data):
    names, maps = [], []

    def create_map(lines):
        ranges = IntervalTree([Interval(0, math.inf)])
        name = lines[0].split()[0]
        for i, line in enumerate(lines[1:]):
            if not line:
                break
            dest, source, size = map(int, line.split())
            ranges.chop(source, source+size)
            ranges[source:source+size] = dest
        return name, ranges, lines[i+2:]

    seeds, data = map(int, data[0].split(': ')[-1].split()), data[2:]

    while len(data):
        name, intervals, data = create_map(data)
        names.append(name)
        maps.append(intervals)

    return list(seeds), maps, names

seeds, intervals, names = day5_maps(data(5))

def day5_1(seeds, data):
    lowest = math.inf
    for seed in seeds:
        for interval in data:
            start, end, dest = next(iter(interval.at(seed)))
            seed = seed if dest is None else (seed-start)+dest
        lowest = min(lowest, seed)
    return lowest

def day5_2(seeds, data):
    seedrange = IntervalTree([Interval(start, start+size) for start, size in zip(seeds[::2], seeds[1::2])])
    seedrange.merge_overlaps()

    for i, interval in enumerate(data):
        newrange = IntervalTree()
        if debug:
            print(names[i], seedrange, interval)
        for (seedstart, seedend, _) in seedrange:
            intersections = interval[seedstart:seedend]
            for (start, end, dest) in intersections:
                newstart = (dest + max(seedstart, start)-start) if dest is not None else seedstart
                newend = (dest + min(seedend, end)-start) if dest is not None else seedend
                newrange.addi(newstart, newend)
        newrange.merge_overlaps()
        seedrange = newrange
        
    lowest = min([loc[0] for loc in seedrange])
    return lowest

day5_1(seeds, intervals), day5_2(seeds, intervals)


(1181555926, 37806486)

In [38]:
data6 = data(6, lambda x: list(map(int, x.split(':')[-1].split())))

def day6(times, distances):
    result = []
    for t, d in zip(times, distances):
        # (t-x)*x = d+1
        # -x**2 +xt -d-1 = 0
        a, b, c = -1, t, -d-1
        det = math.sqrt(b**2 - 4*a*c)
        if det < 0:
            continue
        zeros = (-b+det)/(2*a), (-b-det)/(2*a)
        floor, ceil = math.ceil(min(zeros)), math.floor(max(zeros))
        result.append(ceil-floor+1)
    return functools.reduce(lambda x, y: x*y, result, 1)

day6(*data6), day6(*[[int(''.join(map(str, x)))] for x in data6])

(252000, 36992486)

In [53]:
data7 = data(7, lambda x: x.split())

def card_ord(card, flag=False):
    match card:
        case 'T': return 10
        case 'J': return 1 if flag else 11
        case 'Q': return 12
        case 'K': return 13
        case 'A': return 14
    return int(card)

def score_hand(hand, flag=False):
    cards = [card_ord(card, flag) for card in hand]
    if flag:
        filtered = Counter(hand.replace('J', ''))
        hand = hand.replace('J', filtered.most_common(1)[0][0] if filtered else 'A')

    counts = Counter(hand)
    score = sum([counts[x]**2 for x in counts])
    return [score]+cards

def day7(data, flag=False):
    key = lambda x: score_hand(x[0], flag)
    ordered = sorted(data, key=key)
    return sum((i+1)*int(score) for i, (_, score) in enumerate(ordered))

day7(data7), day7(data7, True)

(248812215, 250057090)

In [69]:
def parse8(data):
    instructions, nodes = data[0], {}
    for line in data[2:]:
        start, l, r = re.findall(r'(.{3}) = \((.{3}), (.{3})\)', line)[0] 
        nodes[start] = (l, r)
    return instructions, nodes

data8 = parse8(data(8))

def day8(instructions, nodes, start, ends):
    count, node = 0, start
    for i in itertools.cycle(instructions):
        node = nodes[node][0 if i == 'L' else 1]
        count +=1
        if ends(node):
            break
    return count

day8(*data8, 'AAA', lambda x: x == 'ZZZ')

18673

In [70]:
def day8_2(instructions, nodes):
    results = []
    for start in filter(lambda x: re.match(r'..A', x), nodes):
        results.append(day8(instructions, nodes, start, lambda x: re.match(r'..Z', x)))
    return math.lcm(*results)

day8_2(*data8)

17972669116327

In [52]:
data9 = data(9, lambda line: [int(x) for x in line.split()])

def determine_steps(arr, i=-1):
    steps = [arr[i]]
    while len(np.unique(arr)) > 1:
        arr = np.diff(arr)
        steps.append(arr[i])
    return steps

def day9(data, part2=False):
    part1, part2 = 0, 0
    for history in data:
        steps = determine_steps(np.array(history)), determine_steps(np.array(history), 0)
        part1 += sum(steps[0])
        part2 += sum(steps[1][::2]) - sum(steps[1][1::2])
    return part1, part2

day9(data9)

(2008960228, 1097)

In [17]:
data10 = np.array(data(10, list))

pipe_map = {
    '|': [(0, -1), (0, 1)],
    '-': [(-1, 0), (1, 0)],
    'L': [(0, -1), (1, 0)],
    'J': [(0, -1), (-1, 0)],
    '7': [(0, 1), (-1, 0)],
    'F': [(0, 1), (1, 0)],
}

def pipe_neighbors(pipe, x, y):
    if pipe not in pipe_map:
        return
    for i in pipe_map[pipe]:
        yield (x+i[0], y+i[1])

def get_neighbors(x, y):
    return [(x-1, y), (x+1, y), (x, y+1), (x, y-1)]

def fix_pipe_start(arr):    
    start_coords = np.where(arr == 'S')
    y, x = start_coords[0][0], start_coords[1][0]

    neighbors = []
    for i, j in get_neighbors(x, y):
        if (x, y) in pipe_neighbors(data10[j, i], i, j):
            neighbors.append((i-x, j-y))
    for pipe in pipe_map.keys():
        if frozenset(pipe_map[pipe]) == frozenset(neighbors):
            arr[y, x] = pipe
    return x, y

def dijkstra(nodes, start, get_neighbors, end_condition=lambda _: False):
    q, visited = deque([(start, 0)]), {}
    while q:
        current, distance = q.popleft()
        if end_condition(current):
            return visited, current
        if current in visited:
            continue
        for node in get_neighbors(nodes, current):
            q.append((node, distance+1))
        visited[current] = distance
    return visited, None

def visualize(arr, vmap):
    visual = arr.copy()
    for x, y in vmap:
        visual[y, x] = vmap[(x, y)]
    print(visual)

def day10_1(data):
    arr = data.copy()
    start = fix_pipe_start(arr)
    visited, _ = dijkstra(data, start, lambda _, coords: pipe_neighbors(arr[coords[1], coords[0]], *coords))
    # visualize(arr, visited)
    return visited

def day10(data):
    visited = day10_1(data)

    claimed = {}
    for y, row in enumerate(data):
        inside = False
        for x, pipe in enumerate(row):
            if (x, y) in visited:
                if pipe in 'LJ|':
                    # If pipe is part of path and is north: taken from https://old.reddit.com/r/adventofcode/comments/18fgddy/2023_day_10_part_2_using_a_rendering_algorithm_to/
                    inside ^= True 
            elif inside:
                claimed[(x, y)] = 'I'
    # visualize(data, claimed)

    return max(visited.values()), len(claimed)

day10(data10)


(6882, 491)

In [66]:
data11 = np.array(data(11, list))

def day11(data, expansions):
    cols, rows = [IntervalTree([Interval(x, x+1) for x in np.where(np.all(data == '.', axis=i))[0]]) for i in range(2)]

    galaxies = np.where(data == '#')

    results = Counter()
    for a, b in itertools.combinations(range(len(galaxies[0])), 2):
        xs, ys = Interval(galaxies[0][a], galaxies[0][b]), Interval(*sorted([galaxies[1][a], galaxies[1][b]]))
        empties = len(cols[ys[0]:ys[1]]) + len(rows[xs[0]:xs[1]])
        distance = ys[1]-ys[0] + xs[1]-xs[0]
        for expansion in expansions:
            results[expansion] += distance + empties * (expansion-1)
    return results

day11(data11, [2, 1000000])

Counter({1000000: 742305960572, 2: 9445168})

In [51]:
def parse12(line):
    springs, conditions = line.split(' ')
    return springs, tuple(map(int, conditions.split(',')))

data12 = data(12, parse12)

@functools.lru_cache()
def fix_records(line, numbers, active=0):
    if not numbers:
        return 0 if '#' in line else 1
    if not line:
        return 0
    n, nt = numbers[0], numbers[1:]
    head, tail = line[0], line[1:]
    if head == '.':
        if active:
            if n > 0:
                return 0
            return fix_records(tail, nt)
        return fix_records(tail, numbers)
    elif head == '#':
        if n == 0:
            return 0
        return fix_records(tail, (n-1, *nt), active+1)
    else:
        return fix_records('.'+tail, numbers, active) + fix_records('#'+tail, numbers, active)

def sum_records(data, n=1):
    result = 0
    for spring, numbers in data:
        result += fix_records('?'.join([spring]*n) + '.', numbers*n)
        # print(spring*n, numbers*n, result)
    return result

def day12(data):
    return sum_records(data), sum_records(data, 5)

day12(data12)

(7857, 28606137449920)

In [52]:
def parse13(lines):
    result = []
    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            return result, lines[i+1:]
        else:
            result.append(list(map(int, line.replace('.', '0').replace('#', '1'))))
    return result, []

def get13():
    patterns = []
    with open(f"./data/day13.txt") as f:
        lines = f.readlines()
        while lines:
            pattern, lines = parse13(lines)
            patterns.append(np.array(pattern))
    return patterns

data13 = get13()

def get_matches(pattern):
    matches = defaultdict(lambda: set())
    for i in range(len(pattern)):
        for j in range(i+1, len(pattern)):
            if np.all(pattern[i]==pattern[j]):
                matches[i].add(j)
                matches[j].add(i)
    return matches

def get_close_matches(pattern):
    for i in range(len(pattern)):
        for j in range(i+1, len(pattern)):
            diffs = np.count_nonzero(pattern[i] != pattern[j])
            if diffs == 1:
                p = pattern.copy()
                saved, p[i] = p[i], p[j]
                yield p, get_matches(p)
                p[i], p[j] = saved, saved
                yield p, get_matches(p)

def check_reflections(pattern, matches):
    length = len(pattern)

    def check_reflection(i):
        distance = min(length, 2*(i+1)) - (i+1)
        if not distance:
            return
        for j in range(distance):
            if (i-j) not in matches[i+1+j]:
                return False
        return distance

    for i in range(length):
        if check_reflection(i):
            yield i+1

def find_reflection(pattern):
    match = get_matches(pattern)
    for h in check_reflections(pattern, match):
        yield h*1j
    t = np.transpose(pattern)
    match = get_matches(t)
    yield from check_reflections(t, match)

def find_smudged_reflection(pattern, disallowed):
    for p, match in get_close_matches(pattern):
        for h in check_reflections(p, match):
            if h != disallowed.imag:
                return h*1j
    for p, match in get_close_matches(np.transpose(pattern)):
        for v in check_reflections(p, match):
            if v != disallowed.real:
                return v

def day13(data):
    def summarize(n):
        return int(n.imag*100+n.real)

    part1, part2 = 0, 0
    for pattern in data:
        disallowed = next(find_reflection(pattern))
        part1 += disallowed
        part2 += find_smudged_reflection(pattern, disallowed)

    return summarize(part1), summarize(part2)

day13(data13)

(31739, 31539)

In [192]:
data14 = np.array(data(14, list))

def tilt(col, direction):
    rock_groups = ''.join(col).split('#')
    tilted = '#'.join(map(lambda x: ''.join(sorted(x, reverse=direction)), rock_groups))
    return list(tilted)
tiltup, tiltdown = lambda x: tilt(x, True), lambda x: tilt(x, False)

def rotate(dish, direction):
    match direction:
        case 0: return np.apply_along_axis(tiltup, 0, dish)
        case 1: return np.apply_along_axis(tiltup, 1, dish)
        case 2: return np.apply_along_axis(tiltdown, 0, dish)
    return np.apply_along_axis(tiltdown, 1, dish)

def calculate_weight(arr):
    weight = 0
    stones = list(reversed(np.count_nonzero(arr == 'O', 1)))
    for i in range(len(arr)):
        weight += (i+1)*stones[i]
    return weight

def day14_1(data):
    return calculate_weight(rotate(data, 0))

def cycle(dish):
    for j in range(4):
        dish = rotate(dish, j)
    return dish

def day14_2(arr, target=1000000000):
    repeated = {}
    for i in range(target):
        arr = cycle(arr)
        if (rs := repeated.setdefault(arr.data.tobytes(), i)) != i:
            i = ((target-rs)//(i-rs))*(i-rs) + rs +1 # potential bug here where before the +1 i is exactly at the target
            break
    for i in range(i, target): # should just figure out the number directly by pulling from repeated
        arr = cycle(arr)

    return calculate_weight(arr)

day14_1(data14), day14_2(data14)

(112773, 98894)

In [42]:
data15 = list(itertools.chain.from_iterable(data(15, lambda x: x.split(','))))

def advent_hash(input):
    value = 0
    for i in input:
        value += ord(i)
        value *= 17
        value %= 256
    return value

def day15(data):
    boxes = defaultdict(lambda: OrderedDict())

    def advent_hash_map(input):
        if input[-1] == '-':     
            label = input[:-1]
            box = advent_hash(label)
            if label in boxes[box]:
                del boxes[box][label]   
        else:
            label, value = input.split('=')
            box = advent_hash(label)
            boxes[box][label] = int(value)

    for input in data:
        advent_hash_map(input)

    result = 0
    for box in boxes:
        for i, label in enumerate(boxes[box]):
            # print(box, i, label)
            result += (box+1) * (i+1) * boxes[box][label]

    return sum([advent_hash(i) for i in data]), result
        
day15(data15)

(501680, 241094)

In [46]:
data16 = np.array(data(16, list))

In [48]:


def day16(data):
    ymax, xmax = data.shape

    def beam(pos, dir):
        space = data[int(pos.imag)][int(pos.real)]
        match space:
            case '-':
                if dir.imag:
                    return [(pos+1, 1), (pos-1, -1)]
            case '|':
                if dir.real:
                    return [(pos+1j, 1j), (pos-1j, -1j)]
            case '/':
                # print('a', pos, dir)
                dir = (-1 if dir.real else 1) * 1j*dir
            case '\\':
                # print('b', pos, dir)
                dir = (-1 if dir.imag else 1) * 1j*dir
        return [(pos+dir, dir)]
    
    visited = set([(0, 1)])
    q = deque([(0, 1)])
    while q:
        pos, dir = q.popleft()
        for p, d in beam(pos, dir):
            if p.real < 0 or p.real >= xmax or p.imag < 0 or p.imag >= ymax:
                continue
            if (p, d) in visited:
                continue
            visited.add((p, d))
            q.append((p, d))

    # print(visited)
    return set([p for p, _ in visited])

lit = day16(data16)

def visualize(arr, vmap):
    visual = arr.copy()
    for i in vmap:
        x, y = int(i.real), int(i.imag)
        visual[y, x] = '#'
    print(visual)

# print(data16, len(lit))
# visualize(data16, lit)
len(lit)


{((59+56j), (-1+0j)), ((100+40j), 1), ((91+79j), (1-0j)), ((46+37j), (1-0j)), ((77+31j), -1), ((109+54j), (1-0j)), ((35+31j), (1-0j)), ((98+48j), (1-0j)), ((47+15j), -1j), ((10+56j), -1), ((51+40j), (1-0j)), ((46+81j), 1j), ((28+31j), (-1+0j)), ((93+37j), 1j), ((19+70j), (-1+0j)), ((60+54j), (1-0j)), ((82+31j), 1j), ((73+70j), 1j), ((49+48j), (1-0j)), ((4+6j), 1), ((28+28j), 1j), ((91+45j), 1j), ((80+95j), -1), ((109+20j), 1j), ((71+104j), 1j), ((76+37j), (1-0j)), ((47+56j), 1), ((89+79j), 1j), ((65+31j), (1-0j)), ((83+6j), 1), ((22+81j), (-1+0j)), ((40+56j), (-1+0j)), ((72+79j), (1-0j)), ((27+37j), (1-0j)), ((90+54j), (1-0j)), ((22+78j), 1j), ((4+28j), (-1+0j)), ((85+95j), 1j), ((76+6j), -1), ((79+48j), (1-0j)), ((70+87j), (1-0j)), ((97+23j), (1-0j)), ((59+81j), (1-0j)), ((65+53j), -1), ((106+37j), (1-0j)), ((56+62j), 1j), ((74+37j), 1j), ((63+87j), (-1+0j)), ((83+51j), 1j), ((30+48j), 1), ((21+87j), (1-0j)), ((36+20j), (-1+0j)), ((7+39j), -1), ((72+45j), 1j), ((70+56j), (-1+0j)), ((1

In [27]:
-1j*-1j

(-1+0j)